In [1]:
%matplotlib inline
from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import numpy as np

In [2]:
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
from path import Path

Using TensorFlow backend.


In [3]:
def create_spectrogram(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = r'C:\Users\simba\Desktop\Audio Classification\Train\Pictures' + name + '.jpg'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [10]:
def create_spectrogram_test(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = name + '.jpg'
    fig.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [5]:
Data_dir=np.array(glob(r"C:\Users\simba\Desktop\Audio Classification\Train\Train\*"))
i=0
r=0
for file in Data_dir[i:]: 
    filename,name = file,file.split('\\')[-1].split('.')[0]
    create_spectrogram(filename,name)
    print(r)
    r=r+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237


In [11]:
Test_dir=np.array(glob(r"C:\Users\simba\Desktop\Audio Classification\Test\Test\*")) 
i=0
r=0
for file in Test_dir[i:]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram_test(filename,name)
    print(r)
    r=r+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


In [14]:
from keras_preprocessing.image import ImageDataGenerator

def append_ext(fn):
    return fn+".jpg"

traindf=pd.read_csv(r'C:\Users\simba\Desktop\Audio Classification\Train\train.csv',dtype=str)
testdf=pd.read_csv(r'C:\Users\simba\Desktop\Audio Classification\Test\test.csv',dtype=str)
traindf["slice_file_name"]=traindf["slice_file_name"].apply(append_ext)
testdf["slice_file_name"]=testdf["slice_file_name"].apply(append_ext)
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [23]:
testdf["slice_file_name"]

0     g120.jpg
1     g121.jpg
2     g122.jpg
3     g123.jpg
4     g124.jpg
5     g125.jpg
6     g126.jpg
7     g127.jpg
8     g128.jpg
9     g129.jpg
10    g130.jpg
11    g131.jpg
12    g132.jpg
13    g133.jpg
14    g134.jpg
15    g135.jpg
16    g136.jpg
17    g137.jpg
18    g138.jpg
19    g139.jpg
20    g140.jpg
21    g141.jpg
22    g142.jpg
23    g143.jpg
24    g144.jpg
25    g145.jpg
26    g146.jpg
27    d121.jpg
28    d122.jpg
29    d123.jpg
30    d124.jpg
31    d125.jpg
32    d126.jpg
33    d127.jpg
34    d128.jpg
35    d129.jpg
36    d130.jpg
37    d131.jpg
38    d132.jpg
39    d133.jpg
40    d134.jpg
41    d135.jpg
42    d136.jpg
43    d137.jpg
44    d138.jpg
45    d139.jpg
46    d140.jpg
47    d141.jpg
48    d142.jpg
49    d143.jpg
50    d144.jpg
51    d145.jpg
52    d146.jpg
Name: slice_file_name, dtype: object

In [18]:
train_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="C:/Users/simba/Desktop/Audio Classification/Train/Pictures",
    x_col="slice_file_name",
    y_col="Class",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

Found 179 validated image filenames belonging to 2 classes.


In [27]:
valid_generator=datagen.flow_from_dataframe(
    dataframe=testdf,
    directory="C:/Users/simba/Desktop/Audio Classification/Test/Pictures",
    x_col="slice_file_name",
    y_col="Class",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

Found 13 validated image filenames belonging to 2 classes.


In [30]:
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_8 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_9 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 31, 31, 64)       

In [32]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5)
#model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID)

Epoch 1/5
5/5 [==============================] - 23s 5s/step - loss: 0.4978 - accuracy: 0.8231 - val_loss: 3.4924 - val_accuracy: 0.0000e+00
Epoch 2/5
5/5 [==============================] - 20s 4s/step - loss: 0.3571 - accuracy: 0.8571 - val_loss: 2.2445 - val_accuracy: 0.1538
Epoch 3/5
5/5 [==============================] - 20s 4s/step - loss: 0.5431 - accuracy: 0.7211 - val_loss: 2.2511 - val_accuracy: 0.1538
Epoch 4/5
5/5 [==============================] - 21s 4s/step - loss: 0.4175 - accuracy: 0.8125 - val_loss: 1.2530 - val_accuracy: 0.1538
Epoch 5/5
5/5 [==============================] - 19s 4s/step - loss: 0.3064 - accuracy: 0.8582 - val_loss: 3.7129 - val_accuracy: 0.1538


In [34]:
test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory="C:/Users/simba/Desktop/Audio Classification/Test/Pictures",
    x_col="slice_file_name",
    y_col=None,
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(64,64))
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

Found 53 validated image filenames.


In [35]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
print(predictions[0:6])

1/1 [==============================] - 3s 3s/step
['Good', 'Good', 'Defective', 'Defective', 'Defective', 'Defective']
